In [1]:
from __future__ import annotations

In [2]:
class Value:
  def __init__(self, data: float, father: Value = None, mother: Value = None) -> None:
    self.name = ""
    self.data = data
    self.father: Value = father
    self.mother: Value = mother
    self.grad = 0 # gradient w.r.t to the target
    self.traversed = False
    self._backward = lambda: None # calculates gradients of parents
    self._zero_grad = lambda: None # zeros out gradients of parents


  def __repr__(self) -> str:
    return f"Value(data={self.data})"
  
  # when backwards called on new node, want to calculate gradients for parents
  # you know the derivative of the parent node wrt to the child node
  
  def __add__(self, other: Value):
    father = self; mother = other
    child = Value(self.data + other.data, father, mother)
    def _backward():
      dchild_dfather = 1
      dchild_dmother = 1
      father.grad += dchild_dfather * child.grad
      mother.grad += dchild_dmother * child.grad
    def _zero_grad():
      father.grad = 0; father.traversed = False
      mother.grad = 0; mother.traversed = False
    child._backward = _backward
    child._zero_grad = _zero_grad
    return child
  
  def __sub__(self, other: Value):
    return self + (-other)
  
  def __mul__(self, other: Value):
    father = self; mother = other
    child = Value(father.data * mother.data, father, mother)
    def _backward():
      dchild_dfather = mother.data
      dchild_dmother = father.data
      father.grad += dchild_dfather * child.grad
      mother.grad += dchild_dmother * child.grad
    def _zero_grad():
      father.grad = 0; father.traversed = False
      mother.grad = 0; mother.traversed = False
    child._backward = _backward
    child._zero_grad = _zero_grad
    return child
  
  def __truediv__(self, other: Value):
    return self * (other ** -1)
  
  def __neg__(self):
    father = self
    child = Value(-father.data, father)
    def _backward():
      dchild_dfather = -1
      father.grad += dchild_dfather * child.grad
    def _zero_grad():
      father.grad = 0; father.traversed = False
    child._backward = _backward
    child._zero_grad = _zero_grad
    return child
  
  def __pow__(self, other: float):
    father = self
    child = Value(father.data ** other, father)
    def _backward():
      dchild_dfather = other * father.data ** (other - 1)
      father.grad += dchild_dfather * child.grad
    def _zero_grad():
      father.grad = 0; father.traversed = False
    child._backward = _backward
    child._zero_grad = _zero_grad
    return child


  def backward(self): # defines this node to be the target. derivative of all nodes in graph wrt to this node. 
    # edge cases to consider: 
    # 1. _backward() should only be called when a node's gradient wrt to target is fully computed. i.e. when all it's children have had their _backward's called. EX: a -> b -> c & a -> c. target order is c, b, a but _backward could be called on a before b. b is a's kid. the gradient of a wrt to c is not fully computed.
    # 2. _backward should only be called once per node. need some kind of cycle detection. a node could have multiple children, so it could be visited multiple times. just tracking count won't be good enough. need to consider if there's a duplicate visit from the same path. 

    # could do reverse dfs. root is last to be marked as traversed. if reversing order, it will be called before any of it's children can be called. dfs traverses all children before marking parent as traversed. reversing this means that parent is marked as traversed before any of it's children. this behavior is desirable for topological sorting. from perspective of root, but can be applied to any node b.c. recursive. 
    # a b c -> c b a. regardless of the path that is taken, b's child, a will always be traversed before b, even if a isn't reached through b. reversing order, a's parents will always be traversed before a.
    # adding a traversed variable, so you don't have to check the list. having extra indicator variables save so much computation. 

    dfs_order = []
    def dfs(node: Value):
      if node.father:
        dfs(node.father)
      if node.mother:
        dfs(node.mother)
      if not node.traversed: # i forgor traversal check
        dfs_order.append(node)
        node.traversed = True
    dfs(self)

    self.grad = 1 # derivative wrt to self is 1
    topological_order = dfs_order[::-1]

    print(dfs_order)
    print(topological_order)

    for node in topological_order:
      node._backward()

  def zero_grad(self): # zeros out gradients of ancestors
    self.grad = 0; self.traversed = False
    def dfs(node: Value):
      if node.father:
        dfs(node.father)
      if node.mother:
        dfs(node.mother)
      node._zero_grad()

In [92]:
a = Value(-6)
b = a**2 - a
b.backward()
a.grad, b.grad

[Value(data=-6), Value(data=36), Value(data=6), Value(data=42)]
[Value(data=42), Value(data=6), Value(data=36), Value(data=-6)]


(-13, 1)

In [96]:
a = Value(2); a.name = "a"
b = Value(3); b.name = "b"
c = a * -b; c.name = "c"
d = c ** 2; d.name = "d"
e = d - c; e.name = "e"
a, b, c, d, e

(Value(data=2), Value(data=3), Value(data=-6), Value(data=36), Value(data=42))

In [98]:
# c.grad = 1 # FUCK ME. THIS WAS THE ISSUE ALL ALONG. Forgot to comment out a line. which then made c gradient 1 higher. i thought the -1 wasn't being applied. it was. 
e.backward()
a.grad, b.grad, c.grad, d.grad, e.grad

# e: 1, d: 1, c: (2*-6 * 1 + -1) = -13, b: 26, a: 39

[]
[]


(39, 26, -13, 1, 1)

In [ ]:
# all in all, my reasoning was actually on point. the only real mistake i made was forgetting to add traversed var. 
# the mistake i spent the most time on was just thinking that the -1 wasn't being applied from sub. what happened was that i forgot to uncomment out code that set c.grad to 1, which then meant that the -1 was being applied, but it was 1 + -13 = 12, which effectively canceled it out. i was not expecting the bug to be here and subsequently wasted a ton of time looking elsewhere. 
# the debugger gave me a massive hint of c.grad being 1 when e's _backward() was called. i was very confused and just was looking for the bug in the wrong place. 
# the way i ended up fixing this was through just running the error in isolation with new variables and seeing that the result was correct. that led me to know that my backprop calculations were actually correct and that the error was somewhere else. 

In [37]:
import random

class Neuron:
  def __init__(self, n_inputs: int):
    self.weights = [Value(random.uniform(-1, 1)) for _ in range(n_inputs)]
    self.bias = Value(random.uniform(-1, 1))

  def __call__(self, inputs: list[Value]):
    return sum([weight * input for weight, input in zip(self.weights, inputs)], self.bias)
  
  def get_weights(self) -> list[Value]:
    return self.weights + [self.bias]
  
  def get_gradients(self) -> list[int]:
    return [weight.grad for weight in self.get_weights()]
  
  # implemented zero grad with dfs from target. 
  # def zero_grad(self):
  #   for weight in self.get_weights():
  #     weight.zero_grad()
  
class Layer:
  def __init__(self, n_inputs: int, n_neurons: int):
    self.neurons = [Neuron(n_inputs) for _ in range(n_neurons)]
  
  def __call__(self, inputs: list[Value]):
    return [neuron(inputs) for neuron in self.neurons]
  
  def get_weights(self) -> list[Value]:
    accumulator_list = []
    for neuron in self.neurons:
      accumulator_list += neuron.get_weights()
    return accumulator_list
  
  def get_gradients(self) -> list[int]:
    accumulator_list = []
    for neuron in self.neurons:
      accumulator_list += neuron.get_gradients()
    return accumulator_list
  
  

class MLP:
  def __init__(self, n_inputs: int, layer_neuron_counts: list[int]):
    self.layers = [Layer(n_inputs, layer_neuron_counts[0])]
    for i in range(1, len(layer_neuron_counts)):
      self.layers.append(Layer(layer_neuron_counts[i-1], layer_neuron_counts[i]))

  def __call__(self, inputs: list[Value]):
    for layer in self.layers:
      inputs = layer(inputs)
    return inputs
  
  def get_weights(self) -> list[Value]:
    accumulator_list = []
    for layer in self.layers:
      accumulator_list += layer.get_weights()
    return accumulator_list
  
  def get_gradients(self) -> list[int]:
    accumulator_list = []
    for layer in self.layers:
      accumulator_list += layer.get_gradients()
    return accumulator_list

In [38]:
n = Neuron(2)
layer = Layer(2, 3)
nn = MLP(2, [3, 4, 5])
len(nn.get_weights()), len(nn.get_gradients())

(50, 50)

In [46]:
xs = [
  [Value(2.0), Value(3.0), Value(-1.0)],
  [Value(3.0), Value(-1.0), Value(0.5)],
  [Value(0.5), Value(1.0), Value(1.0)],
  [Value(1.0), Value(1.0), Value(-1.0)]
]
ys = [[Value(1.0)], [Value(-1.0)], [Value(-1.0)], [Value(1.0)]]

In [47]:
mlp = MLP(3, [4, 4, 1])
preds = [mlp(x) for x in xs]

In [48]:
loss = Value(0)
for pred, target in zip(preds, ys):
  example_squared_error = Value(0)
  for idx in range(len(pred)): # compare each element
    example_squared_error += (pred[idx] - target[idx]) ** 2
  loss += example_squared_error

loss

Value(data=18.47478973109362)

In [60]:
loss.zero_grad()
loss.backward()
weights = mlp.get_weights()
gradients = mlp.get_gradients()
mlp.get_gradients()

[Value(data=0), Value(data=0), Value(data=1.7613822362361928), Value(data=1.2701192112277044), Value(data=-0.34137882614086834), Value(data=0.9287403850868361), Value(data=1.0118053528104518), Value(data=1.940545737897288), Value(data=0.21829016290834333), Value(data=0.6188437091520453), Value(data=1.4677240672794283), Value(data=1.1296439575595607), Value(data=-3.84749352902505), Value(data=-2.7178495714654893), Value(data=0.34485799421340235), Value(data=-2.372991577252087), Value(data=0.11766455768644887), Value(data=0.7365082668384942), Value(data=-1.969093771370893), Value(data=-1.2956473388895753), Value(data=0.7137540816476307), Value(data=-0.5818932572419446), Value(data=-0.5346138377350409), Value(data=-1.1165070949769855), Value(data=-0.4754274273193303), Value(data=0.26108083951916394), Value(data=0.5587857834852374), Value(data=0.38556285578401295), Value(data=1.442981505424242), Value(data=1.828544361208255), Value(data=-0.6129934271003112), Value(data=1.2155509341079438),

[-12.555120244978937,
 -30.513250480870735,
 13.018184424466286,
 -11.627891987858188,
 1.3788447150911207,
 11.041757338568871,
 -8.441359045621882,
 2.3990763614763564,
 -0.22177753984287807,
 4.384008665880717,
 -4.251655098536462,
 0.5127808163118431,
 -3.885052771745498,
 -10.088269916813191,
 4.631254396705464,
 -3.6926220855220624,
 16.60946495321727,
 10.573257499112716,
 -1.8856658255812093,
 3.36188616771466,
 -8.552967819488272,
 13.606960994951182,
 8.376376521628359,
 -1.6193365864879496,
 3.517593717294553,
 -6.517449799499002,
 1.338181361971038,
 0.8079989391588179,
 -0.2253566211861244,
 1.2076966669712466,
 -0.23741929076133905,
 13.536867102168296,
 7.758041576558073,
 -1.7705402597119038,
 5.21595019871387,
 -5.418201015805006,
 25.776613960996393,
 7.95261401308304,
 3.1861078585558773,
 9.13439342066053,
 -9.799725244971718]

In [61]:
learning_rate = 0.01
for weight in weights:
  weight.data -= learning_rate * weight.grad

In [62]:
preds = [mlp(x) for x in xs]

loss = Value(0)
for pred, target in zip(preds, ys):
  example_squared_error = Value(0)
  for idx in range(len(pred)): # compare each element
    example_squared_error += (pred[idx] - target[idx]) ** 2
  loss += example_squared_error

loss

Value(data=2.0657631306157977)

In [63]:
# not normalizing loss seems to make the gradients larger. checks out since the height of the loss function becomes much larger in general. loss function landscape becomes far steeper. input space does not scale in comparison. you would have to change the learning rate to be much smaller to compensate for the larger gradients. 
# if you change your dataset/batch size, you'd have to change learning rate. kinda fiddly and annoying. makes far more sense to just normalize and change lr much less. 